In [ ]:
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import stanza

from plotly.subplots import make_subplots
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from tqdm import tqdm
from yellowbrick.cluster import KElbowVisualizer

tqdm.pandas()

In [ ]:
# We allow the program to read in saved tables in order to reduce running time, which can be quite large.
recalculate_all = True

In [ ]:
base_path = '.'
spreadsheets_path = 'Movie_spreadsheets'
data_path = os.path.join(base_path, spreadsheets_path)
intermediate_path = os.path.join(base_path, 'intermediate')

# Data Processing

In [ ]:
def split_lines_with_character_tags(df):
    df = df.sample(frac=1)
    df = df.reset_index(drop=True)
    # Remove text which are labeled character names but which are not all caps (which often indicate stage directions)
    df['line #'] = df.index
    df['line'] = df['line'].str.replace('\n', ' ')
    df = df[df['character'].str.isupper()]
    
    # Remove parentheticals such as "(CONTD)" or "(V.O.)"
    df['character'] = df['character'].str.replace(r"\(.+\)", '').str.strip()
    df['original character'] = df['character']
    df['line'] = df['line'].str.replace(r'\(cont.+d\)', '', flags=re.IGNORECASE)
    df = df[df['character'].str.strip().str.len() != 0]


    for character in df['original character'].unique():
        # If the name of a character (ONLY if in all caps, as is expected of character headers)
        # appears in a section of speech, separate it into another section of speech.
        # This is because we frequently observed one character's speech being contaminated with another's, e.g.:
            # SAM: Hi there how's it going? ALICE: I'm doing well
            # SAM: Glad to hear it.
        while len(df[df['line'].str.contains(re.escape(character))]) > 0:
            new_rows = df[df['line'].str.contains(character)].copy()
            new_rows.loc[:,'line'] = new_rows['line'].str.split(character).apply(lambda x: x[1])
            df.loc[:,'line'] = df['line'].str.split(character).apply(lambda x: x[0])
            new_rows['character'] = character
            new_rows.loc[:,'line #'] += .001
            df = df.append(new_rows)
            df = df.sort_values(by='line #').reset_index(drop=True)
            df.loc[:, 'line #'] = df.index

    df = df.drop('original character', axis=1)        
    df['character'] = df['character'].str.replace(r'[^A-z]','')

    df ['line'] = df['line'].str.replace('\r', '')
    df['line'] = df['line'].str.replace('\\\'', '\'')
    df = df[df['line'].str.strip().str.len() != 0]
    df = df[df['character'].str.strip().str.len() != 0]
    df = df.dropna()
    return df

In [ ]:
# Silence chained assignment warnings which are incorrect
pd.options.mode.chained_assignment = None

In [ ]:
if recalculate_all:
    # For every movie in the dataset, run the cleaning function
    df = pd.DataFrame()
    movie_id = 0
    for file in tqdm(sorted(os.listdir(data_path))):
        if file.endswith('.csv'):
            movie_title = file.replace('.csv', '')
            # print(movie_title)
            individual_df = pd.read_csv(os.path.join(data_path, file), index_col=0)
            # print(file)
            # print(individual_df.head())
            individual_df['movie_id'] = movie_id
            movie_id += 1
            individual_df['movie_title'] = movie_title
            individual_df = split_lines_with_character_tags(individual_df)
            df = df.append(individual_df)

    # Weird problem in the labeling of Black Panther
    df = df[~df['character'].isin(['T', 'A', 'N'])]
    df = df.reset_index(drop=True)

In [ ]:
if recalculate_all:
    # Find the number of lines per character
    character_counts = pd.DataFrame(df.groupby(['character', 'movie_title'])['line #'].count())
    character_counts.hist()

In [ ]:
if recalculate_all:
    top_character_counts = character_counts[character_counts['line #'] >= 60]
    bottom_character_counts = character_counts[character_counts['line #'] <= 60]
    top_character_counts.hist()

In [ ]:
if recalculate_all:
    # Drop any characters who have less than 60 lines
    for character, movie in tqdm(bottom_character_counts.index):
        df = df[~((df['character'] == character) & (df['movie_title'] == movie))]

# Sentiment Scoring

We use the stanford Stanza library to assign a sentiment score to every line. The library assigns sentiment scores for each sentence (0, 1, or 2). If a line contains multiple sentences, we take the average (mean) sentiment of the line.

In [ ]:
if recalculate_all:
    stanza.download('en')
    nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment', use_gpu=False)

    def sentiment_score(text):
        doc = nlp(text)
        scores = []
        for i, sentence in enumerate(doc.sentences):
            scores.append(sentence.sentiment)
        return np.mean(scores)
    
    df['sentiment'] = df['line'].progress_apply(sentiment_score)

In [ ]:
# Get a new line ID which gives the position of the line in the whole movie, not simply in the character's lines
# This enables us to know, for example, whether a line occurred in the beginning or end of the movie
# rather than just in the beginning or end of a character's own dialog
if recalculate_all:
    df['movie_length'] = 0
    for movie_id in tqdm(df['movie_id']):
        movie_length = df.loc[df['movie_id'] == movie_id]['line #'].max()
        df.loc[df['movie_id'] == movie_id, 'movie_length'] = movie_length
    df['relative_line_id'] = df['line #'] / df['movie_length']

In [ ]:
# Assign an index which contains both the character and movie name. 
# If we did not elect to recalculate everything, we now load in the existing sentiment scores
if recalculate_all:
    df['combined_name'] = df['movie_title'].str.cat(df['character'], sep='_')
    df.to_csv(os.path.join(intermediate_path, 'movie_df_shuffle.csv'))
else:
    df = pd.read_csv(os.path.join(intermediate_path, 'movie_df_shuffle.csv'))

In [ ]:
if recalculate_all:
    expanded_movie_size = df['movie_length'].max()
    sentiment_df = np.ones(shape=(len(df['combined_name'].unique()), expanded_movie_size))
    sentiment_df = pd.DataFrame(sentiment_df)
    sentiment_df.index = df['combined_name'].unique()

    # Arbitrarily mark cells whose sentiment scores did not originate from the movie dataframe.
    # True sentiment scores cannot be less than 0
    sentiment_df = sentiment_df * -10
    
    # We standardize the length of movies by assigning each line to the index of its relative position in the movie 
    # (e.g. 66% of the way through) to its position in the expanded standardized movie length
    # e.g. if the longest movie had 5000 lines, a line at 66% of the way through its own movie
    # would be assigned to position 3300 in its new "relative" movie length
    for row in df.iterrows():
        sentiment_df.loc[row[1]['combined_name'], np.round(row[1]['relative_line_id'] * (expanded_movie_size - 1))] = row[1]['sentiment']

In [ ]:
np.random.seed(42)

def expand_row(row):
    # To allow comparison, we needed to fill in the empty values where character was not speaking. We elected to 
    # maintain the character's previous sentiment for all non-speaking lines, adding some gaussian noise
    std = row[row != -10].std()
    if row[0] == -10:
        first_nonempty_index = row[row != -10].index[0]
        first_nonempty_value = row[row != -10].loc[first_nonempty_index]
        gaussian_noise = np.random.normal(0, std / 5, size=first_nonempty_index)
        row[:first_nonempty_index] = first_nonempty_value + gaussian_noise
    for i in range(len(row)):
        if row[i] == -10:
            gaussian_noise = np.random.normal(0, std / 5)
            row[last_index:i+1] = last_val + gaussian_noise
        last_index = i
        last_val = row[i]
    # Standardize each character's arc to have a mean of 0 and a standard deviation of 1
    std = row[row != -10].std()
    row = (row - row.mean()) / std

    return row
        
if recalculate_all:
    sentiment_df = sentiment_df.progress_apply(expand_row, axis=1)
    sentiment_df.to_csv(os.path.join(intermediate_path, 'sentiment_df_shuffle.csv'))
else:
    sentiment_df = pd.read_csv(os.path.join(intermediate_path, 'sentiment_df_shuffle.csv'), index_col=0)

# PCA

We perform principal component analysis for dimensionality reduction to inspect for visible clusters

In [ ]:
pca = PCA(random_state=42)

In [ ]:
# For PCA, we use a very small smoothing window (3 timepoints across) to reduce noise
rolling_df = sentiment_df.rolling(3, axis=1).mean().dropna(axis=1)

In [ ]:
pca_df = pd.DataFrame(pca.fit_transform(rolling_df))
pca_df.index = rolling_df.index

Plot the standard deviation of all movies across time

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=sentiment_df.columns, y=sentiment_df.std(), mode='lines'))
# fig = go.Scatter()
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="σ of Sentiment",
                  )
fig.show()

#### Plot the average sentiment over time for all characters

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=sentiment_df.columns, y=sentiment_df.mean().rolling(30).mean(), mode='lines'))
# fig = go.Scatter()
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="Sentiment",
                  ) 
fig.show()

All characters' first two principal components visualized

In [ ]:
fig = px.scatter(pca_df.reset_index(), 0, 1, hover_name='index', width=650, height=500)
fig.update_layout(
    xaxis_title="PC0",
    yaxis_title="PC1",

)
fig.show()

#### First Principal Component

In [ ]:
midpoint = len(sentiment_df) // 2

def melt_sentiment_df(df):
    return pd.melt(df.reset_index(), id_vars='index').rename({'index': 'Character', 'variable': 'Time', 'value': 'Sentiment'}, axis=1)

In [ ]:

temp_df = sentiment_df.loc[pca_df.sort_values(0).iloc[-5:].index]
fig = go.Figure()
for index in temp_df.index:
    fig.add_trace(go.Scatter(x=temp_df.columns, y=temp_df.loc[index],
                    mode='lines'))
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="Sentiment",
                  ) 
fig.show()

In [ ]:
temp_df = sentiment_df.loc[pca_df.sort_values(0).iloc[midpoint-2:midpoint+3].index]
fig = go.Figure()
for index in temp_df.index:
    fig.add_trace(go.Scatter(x=temp_df.columns, y=temp_df.loc[index],
                    mode='lines'))
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="Sentiment",
                  ) 
fig.show()

In [ ]:
temp_df = sentiment_df.loc[pca_df.sort_values(0).iloc[:5].index]
fig = go.Figure()
for index in temp_df.index:
    fig.add_trace(go.Scatter(x=temp_df.columns, y=temp_df.loc[index],
                    mode='lines'))
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="Sentiment",
                  ) 
fig.show()

#### Second Principal Component

In [ ]:
temp_df = sentiment_df.loc[pca_df.sort_values(1).iloc[-5:].index]
fig = go.Figure()
for index in temp_df.index:
    fig.add_trace(go.Scatter(x=temp_df.columns, y=temp_df.loc[index],
                    mode='lines'))
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="Sentiment",
                  ) 
fig.show()

In [ ]:
import plotly.graph_objects as go

temp_df = sentiment_df.loc[pca_df.sort_values(1).iloc[midpoint-2:midpoint+3].index]
fig = go.Figure()
for index in temp_df.index:
    fig.add_trace(go.Scatter(x=temp_df.columns, y=temp_df.loc[index],
                    mode='lines'))
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="Sentiment",
                  )
fig.show()

In [ ]:
temp_df = sentiment_df.loc[pca_df.sort_values(1).iloc[:5].index]
fig = go.Figure()
for index in temp_df.index:
    fig.add_trace(go.Scatter(x=temp_df.columns, y=temp_df.loc[index],
                    mode='lines'))
fig.update_layout(showlegend=False, 
                    width=650, 
                    height=500,
                    xaxis_title="Time",
                    yaxis_title="Sentiment",
                  ) 
fig.show()

In [ ]:
# from sklearn.manifold import TSNE

# tsne = TSNE( learning_rate='auto', init='pca')
# tsne_df = pd.DataFrame(tsne.fit_transform(sentiment_df.rolling(200, min_periods=1, axis=1).mean().dropna(axis=1)))
# tsne_df['ix'] = sentiment_df.index

# px.scatter(tsne_df, x=0, y=1, hover_name='ix').show()

# Clustering

To reduce noise, we will use a sliding, smoothing window of length 500.

In [ ]:
rolling_window = 500

In [ ]:
rolling_sentiment_df = sentiment_df.rolling(rolling_window, axis=1, center=True).mean().dropna(axis=1)

As one distance metric, we use dynamic time warping, which should be invariant to small translations in the exact timing of patterns

In [ ]:
if recalculate_all:
    from tslearn.metrics import cdist_dtw as dtw
    distances = dtw(sentiment_df, n_jobs=-1, verbose=50)

    dtw_df = pd.DataFrame(distances)
    dtw_df.index = sentiment_df.index
    dtw_df.columns = sentiment_df.index

    dtw_df.to_csv(os.path.join(intermediate_path, 'distances_shuffle.csv'))
else:
    dtw_df = pd.read_csv(os.path.join(intermediate_path, 'distances_shuffle.csv'), index_col=0)

# Clustering

## Setup

In [ ]:
def find_subplot_dimensions(n):
    return (int(np.ceil(n / 3)), 3)

In [ ]:
# We settled on 9 clusters as a minimum of the optimal clusterings from all of our models
n_clusters=9

def cluster(data, model):
    """
    This function helps to calculate and visualize clusters using whatever
    clustering model is passed in
    """
    visualizer = KElbowVisualizer(model, k=(2,30), timings=False)
    visualizer.fit(data)        # Fit data to visualizer
    visualizer.show()
    # n_clusters = visualizer.elbow_value_
    
    model.set_params(n_clusters=n_clusters)
    cluster_df = pd.Series(model.fit_predict(data), index=sentiment_df.index)
    cluster_df = cluster_df.rename({0: 'cluster'})
    clusters = cluster_df.unique()
    clusters.sort()
    rows, cols = find_subplot_dimensions(n_clusters)
    outer_fig = make_subplots(
        rows=rows, 
        cols=cols, 
        shared_xaxes='all', 
        shared_yaxes='all',
        x_title='Time',
        y_title='Sentiment',
        horizontal_spacing=0.03,
        vertical_spacing=0.05,
        subplot_titles=(["Cluster {}".format(i) for i in range(n_clusters)])
    )
    outer_fig.update_layout(showlegend=False, 
                    width=(cols+1)*200, 
                    height=(rows+1)*200,
                )

    
    plotted_count = 0
    
    for i in range(1, rows+1):
        for j in range(1, cols+1):
            if plotted_count < n_clusters:
                indices = cluster_df[cluster_df == plotted_count].dropna().index
                temp_df = rolling_sentiment_df.loc[indices]
                for index in temp_df.index:
                    outer_fig.append_trace(
                        go.Scatter(
                            x=temp_df.columns, y=temp_df.loc[index],
                            mode='lines',
                            # text=index, 
                            marker=dict(
                                color='rgba(135, 206, 250, 0.8)',
                            ),
                            hoverinfo='none',
                            ), 
                        i, 
                        j)
                outer_fig.append_trace(
                    go.Scatter(
                        x=temp_df.columns, y=temp_df.mean(),
                        mode='lines',
                        marker=dict(
                            color='rgba(128, 128, 128, .8)',
                        ),
                    ), 
                    i, 
                    j
                )
                plotted_count += 1
    outer_fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(
        title=dict(
            text='Sentiment'
        )
    )
    outer_fig.show()
    return cluster_df

In [ ]:
if not os.path.exists(os.path.join(base_path, 'clusters_shuffle.csv')):
    clusters_df = pd.DataFrame(index=sentiment_df.index)
else:
    clusters_df = pd.read_csv(os.path.join(base_path, 'clusters_shuffle.csv'), index_col=0)

### Euclidean

In [ ]:
model = AgglomerativeClustering()
clusters_df['euclidean_sentiment'] = cluster(rolling_sentiment_df, model)

### Manhattan

In [ ]:
model = AgglomerativeClustering(linkage='complete', affinity='manhattan')
clusters_df['manhattan_sentiment'] = cluster(rolling_sentiment_df, model)

## On the differences between each timepoint

### Setup

In [ ]:
diff_df = rolling_sentiment_df.diff(axis=1).dropna(axis=1)

### Euclidean

In [ ]:
model = AgglomerativeClustering()
clusters_df['euclidean_diff'] = cluster(diff_df, model)

### Manhattan

In [ ]:
model = AgglomerativeClustering(linkage='complete', affinity='manhattan')
clusters_df['manhattan_diff'] = cluster(diff_df, model)

## On dynamic time warp distances

In [ ]:
model = AgglomerativeClustering(affinity='precomputed', linkage='complete')
clusters_df['dtw_sentiment'] = cluster(dtw_df, model)

## Gold standard

We read in the gold standard clusterings and standardize their formatting

In [ ]:
gold_standard_df = pd.read_json(os.path.join(base_path, 'tvtropes.clusters.json')).T.drop('id', axis=1)
gold_standard_df.sort_values('movie')
gold_standard_df.to_csv(os.path.join(base_path, 'gold_standard.csv'))

In [ ]:
if not 'gold_standard_cluster' in clusters_df.columns:
    clusters_df['gold_standard_cluster'] = '0' 

In [ ]:
for loc in clusters_df.index:
    if not pd.isna(clusters_df.loc[loc, 'gold_standard_cluster']) and clusters_df.loc[loc, 'gold_standard_cluster'] == '0':
        mapping = input(loc)
        clusters_df.loc[loc, 'gold_standard_cluster'] = mapping
        clusters_df.to_csv(os.path.join(base_path, 'clusters_shuffle.csv'))
    else:
        pass

There are 77 characters who appear in both datasets (have both inferred and gold standard clusterings):

In [ ]:
clusters_df[~clusters_df['gold_standard_cluster'].isna()]

#### Cluster Purities

According to Bamman et al. 2013 cluster purity metric

In [ ]:
purity_df = clusters_df.copy().dropna()
value_counts = purity_df.value_counts('gold_standard_cluster')
supported_clusters = value_counts.where(value_counts > 1).dropna().index.to_list()
purity_df[purity_df['gold_standard_cluster'].isin(supported_clusters)].value_counts('gold_standard_cluster')

clusterings = purity_df.columns.to_list()
clusterings.remove('gold_standard_cluster')

purities = {}

for clustering in clusterings:
    purity = pd.crosstab(purity_df[clustering], purity_df['gold_standard_cluster']).max().sum() / len(purity_df)
    purities[clustering] = purity

In [ ]:
purities

#### Baseline cluster purities

Assigning each character randomly to a cluster based on the proportion of that cluster in the original clustering (According to Bamman et al. 2013 baseline methodology)

In [ ]:
cluster_distributions_df = pd.DataFrame([clusters_df[clustering].value_counts().sort_index() / len(clusters_df) for clustering in clusterings])

In [ ]:
np.random.seed(0)
purity_baseline_df = purity_df.copy()
for clustering in clusterings:
    purity_baseline_df[clustering] = np.random.choice(n_clusters, size=len(purity_baseline_df), p=cluster_distributions_df.loc[clustering])
    
baseline_purities = {}

for clustering in clusterings:
    purity = pd.crosstab(purity_baseline_df[clustering], purity_baseline_df['gold_standard_cluster']).max().sum() / len(purity_baseline_df)
    baseline_purities[clustering] = purity

In [ ]:
baseline_purities

#### Visual inspection of gold-standard clusters

In [ ]:
stoners = purity_df[purity_df['gold_standard_cluster'] == 'stoner'].index.to_list()
print(stoners)
fig = go.Figure()
for stoner in stoners:
    temp_df = rolling_sentiment_df.loc[stoner]
    fig.add_trace(go.Scatter(
            x=temp_df.index, y=temp_df.values,
            mode='lines',
            # text=index, 
            marker=dict(
                color='rgba(135, 206, 250, 0.8)',
            ),
            hoverinfo='none',
            ))
fig.show()

In [ ]:
purity_df[purity_df['gold_standard_cluster'].isin(supported_clusters)].value_counts('gold_standard_cluster')

In [ ]:
stoners = purity_df[purity_df['gold_standard_cluster'] == 'stoner'].index.to_list()
print(stoners)
fig = go.Figure()
for stoner in stoners:
    temp_df = rolling_sentiment_df.loc[stoner]
    fig.add_trace(go.Scatter(
            x=temp_df.index, y=temp_df.values,
            mode='lines',
            # text=index, 
            marker=dict(
                color='rgba(135, 206, 250, 0.8)',
            ),
            hoverinfo='none',
            ))
fig.show()

In [ ]:
stoners = purity_df[purity_df['gold_standard_cluster'] == 'crazy_jealous_guy'].index.to_list()
print(stoners)
fig = go.Figure()
for stoner in stoners:
    temp_df = rolling_sentiment_df.loc[stoner]
    fig.add_trace(go.Scatter(
            x=temp_df.index, y=temp_df.values,
            mode='lines',
            # text=index, 
            marker=dict(
                color='rgba(135, 206, 250, 0.8)',
            ),
            hoverinfo='none',
            ))
fig.show()

In [ ]:
stoners = purity_df[purity_df['gold_standard_cluster'] == 'stupid_crooks'].index.to_list()
print(stoners)
fig = go.Figure()
for stoner in stoners:
    temp_df = rolling_sentiment_df.loc[stoner]
    fig.add_trace(go.Scatter(
            x=temp_df.index, y=temp_df.values,
            mode='lines',
            # text=index, 
            marker=dict(
                color='rgba(135, 206, 250, 0.8)',
            ),
            hoverinfo='none',
            ))
fig.show()

In [ ]:
stoners = purity_df[purity_df['gold_standard_cluster'] == 'loveable_rogue'].index.to_list()
print(stoners)
fig = go.Figure()
for stoner in stoners:
    temp_df = rolling_sentiment_df.loc[stoner]
    fig.add_trace(go.Scatter(
            x=temp_df.index, y=temp_df.values,
            mode='lines',
            # text=index, 
            marker=dict(
                color='rgba(135, 206, 250, 0.8)',
            ),
            hoverinfo='none',
            ))
fig.show()